In [1]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer
import pandas as pd

In [20]:
# path_to_pdf = "/Users/janek/Coding/PersonalProjects/playground/pdfGPT/assets/animalFarmSummaryAndAnalysis.pdf"
path_to_pdf = "/Users/janek/Coding/PersonalProjects/playground/pdfGPT/assets/orwellanimalfarm.pdf"

fp = open(path_to_pdf, 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)

if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
    
rsrcmgr = PDFResourceManager()

device = PDFDevice(rsrcmgr)

laparams = LAParams()

device = PDFPageAggregator(rsrcmgr, laparams=laparams)

In [27]:
interpreter = PDFPageInterpreter(rsrcmgr, device)

pdf_contents_df = pd.DataFrame(columns = ["x1", "y1", "x2", "y2", "content", "page"])

def parse_obj(lt_objs, pdf_contents_df, pg):
    st = ""
    # loop over the object list
    for obj in lt_objs:
        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            txt = obj.get_text().replace('\n', '')
            #print(txt)
            st = txt.split("(")
            st = ["("+i for i in st[1:]]
            text_val = ""
            for i in st:
                text_str = i.strip()    

                if 'cid' in text_str.lower():
                    text_str = text_str.strip('(')
                    text_str = text_str.strip(')')
                    ascii_num = text_str.split(':')[-1]
                    ascii_num = int(ascii_num)
                    text_val += str(chr(ascii_num))  # 66 = 'B' in ascii
            print(text_val)
            #print(text_val)
            #print("%6d, %6d,%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.bbox[2], obj.bbox[3], obj.get_text().replace('\n', '')))
            print("%6d, %6d,%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.bbox[2], obj.bbox[3], text_val))
            pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)

            st += obj.get_text().replace('\n', '')
        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            print("elif")
            pdf_contents_df = parse_obj(obj._objs, pdf_contents_df, pg)

    #print(st) 
    return pdf_contents_df

In [28]:
i = 0
page = next(PDFPage.create_pages(document))
interpreter.process_page(page)
layout = device.get_result()
pdf_contents_df = parse_obj(layout._objs, pdf_contents_df, i)


    18,    778,   141,    788, 

   221,    668,   390,    696, 

    68,    632,   217,    653, 

   104,    548,   109,    569, 

   104,    438,   507,    533, 

   104,    268,   504,    427, 

   104,     82,   507,    257, 

    18,      2,   341,     12, 


/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/316557577.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/316557577.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/316557577.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [23]:
pdf_contents_df

,x1,y1,x2,y2,content,page
0,18.0000,778.1370,141.5250,788.5410,,0
1,221.3533,668.4031,390.6493,696.9631,,0
2,68.0000,632.3053,217.0580,653.7253,,0
3,104.0000,548.3053,109.0040,569.7253,,0
4,104.0000,438.3379,507.5920,533.9649,,0
5,104.0000,268.3331,504.4700,427.9625,,0
6,104.0000,82.3277,507.9560,257.9577,,0
7,18.0000,2.1372,341.1450,12.5412,,0


In [24]:
i = 0
for page in PDFPage.create_pages(document):
    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()
    print(i)
    i += 1
    # extract text from this object
    pdf_contents_df = parse_obj(layout._objs, pdf_contents_df, i)
    print(pdf_contents_df)
    if i >100:
        break

0
    18,    778,   141,    788, 
   221,    668,   390,    696, 
    68,    632,   217,    653, 
   104,    548,   109,    569, 
   104,    438,   507,    533, 
   104,    268,   504,    427, 
   104,     82,   507,    257, 
    18,      2,   341,     12, 
          x1        y1        x2        y2 content page
0    18.0000  778.1370  141.5250  788.5410            0
1   221.3533  668.4031  390.6493  696.9631            0
2    68.0000  632.3053  217.0580  653.7253            0
3   104.0000  548.3053  109.0040  569.7253            0
4   104.0000  438.3379  507.5920  533.9649            0
5   104.0000  268.3331  504.4700  427.9625            0
6   104.0000   82.3277  507.9560  257.9577            0
7    18.0000    2.1372  341.1450   12.5412            0
8    18.0000  778.1370  141.5250  788.5410            1
9   221.3533  668.4031  390.6493  696.9631            1
10   68.0000  632.3053  217.0580  653.7253            1
11  104.0000  548.3053  109.0040  569.7253            1
12  104.0000  

/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

7
    18,    778,   141,    788, 
   104,    406,   507,    725, 
   104,    156,   502,    395, 
   104,     98,   487,    145, 
   104,     72,   468,     87, 
    18,      2,   341,     12, 
          x1        y1        x2        y2 content page
0    18.0000  778.1370  141.5250  788.5410            0
1   221.3533  668.4031  390.6493  696.9631            0
2    68.0000  632.3053  217.0580  653.7253            0
3   104.0000  548.3053  109.0040  569.7253            0
4   104.0000  438.3379  507.5920  533.9649            0
..       ...       ...       ...       ...     ...  ...
82  104.0000  406.3295  507.5920  725.9649            8
83  104.0000  156.3217  502.6360  395.9541            8
84  104.0000   98.3211  487.8100  145.9463            8
85  104.0000   72.3217  468.4900   87.9457            8
86   18.0000    2.1372  341.1450   12.5412            8

[87 rows x 6 columns]
8
    18,    778,   141,    788, 
   104,    710,   144,    725, 
   104,    652,   491,    699, 
   104,    62

/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

15
    18,    778,   141,    788, 
   104,    454,   507,    725, 
   104,    140,   504,    443, 
   104,     98,   501,    129, 
   104,     72,   492,     87, 
    18,      2,   346,     12, 
           x1        y1        x2        y2 content page
0     18.0000  778.1370  141.5250  788.5410            0
1    221.3533  668.4031  390.6493  696.9631            0
2     68.0000  632.3053  217.0580  653.7253            0
3    104.0000  548.3053  109.0040  569.7253            0
4    104.0000  438.3379  507.5920  533.9649            0
..        ...       ...       ...       ...     ...  ...
144  104.0000  454.3313  507.9560  725.9649           16
145  104.0000  140.3211  504.1200  443.9559           16
146  104.0000   98.3211  501.3340  129.9457           16
147  104.0000   72.3217  492.7940   87.9457           16
148   18.0000    2.1372  346.1490   12.5412           16

[149 rows x 6 columns]
16
    18,    778,   141,    788, 
   104,    422,   507,    725, 
   104,    252,   501,    411,

...

/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

62
    18,    778,   141,    788, 
   104,    534,   507,    725, 
   104,    428,   504,    523, 
   104,    258,   506,    417, 
   104,    120,   507,    247, 
    18,      2,   346,     12, 
           x1        y1        x2        y2 content page
0     18.0000  778.1370  141.5250  788.5410            0
1    221.3533  668.4031  390.6493  696.9631            0
2     68.0000  632.3053  217.0580  653.7253            0
3    104.0000  548.3053  109.0040  569.7253            0
4    104.0000  438.3379  507.5920  533.9649            0
..        ...       ...       ...       ...     ...  ...
485  104.0000  534.3343  507.9980  725.9649           63
486  104.0000  428.3319  504.8620  523.9589           63
487  104.0000  258.3271  506.6260  417.9565           63
488  104.0000  120.3235  507.6200  247.9517           63
489   18.0000    2.1372  346.1490   12.5412           63

[490 rows x 6 columns]
63
    18,    778,   141,    788, 
   104,    696,   116,    717, 
   104,    618,   505,    681,

/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

69
    18,    778,   141,    788, 
   104,    614,   502,    725, 
   104,    540,   504,    603, 
   104,    450,   506,    529, 
   104,    248,   502,    439, 
   104,     78,   507,    237, 
    18,      2,   346,     12, 
           x1        y1        x2        y2 content page
0     18.0000  778.1370  141.5250  788.5410            0
1    221.3533  668.4031  390.6493  696.9631            0
2     68.0000  632.3053  217.0580  653.7253            0
3    104.0000  548.3053  109.0040  569.7253            0
4    104.0000  438.3379  507.5920  533.9649            0
..        ...       ...       ...       ...     ...  ...
535  104.0000  540.3361  504.4980  603.9619           70
536  104.0000  450.3343  506.0520  529.9607           70
537  104.0000  248.3283  502.1180  439.9589           70
538  104.0000   78.3235  507.3400  237.9529           70
539   18.0000    2.1372  346.1490   12.5412           70

[540 rows x 6 columns]
70
    18,    778,   141,    788, 
   104,    678,   490,    725,

/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_contents_df = pdf_contents_df.append({"x1" : obj.bbox[0], "y1" : obj.bbox[1], "x2" : obj.bbox[2], "y2" : obj.bbox[3], "content" : text_val, "page" : pg}, ignore_index = True)
/var/folders/70/xy86ymm940796lf7988yvm9h0000gn/T/ipykernel_64438/3589494632.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat